In [455]:
import pandas as pd
import io
import requests
import numpy as np

In [457]:
#URL CSV to dataframe
url="https://gist.githubusercontent.com/bobbae/b4eec5b5cb0263e7e3e63a6806d045f2/raw/279b794a834a62dc108fc843a72c94c49361b501/data.csv"
response=requests.get(url).content
data_file=pd.read_csv(io.StringIO(response.decode('utf-8')))

In [459]:
# Check our data frame 
data_file.head()

,Year,Rank,Company,Revenue (in millions),Profit (in millions)
0,1955,1,General Motors,9823.5,806
1,1955,2,Exxon Mobil,5661.4,584.8
2,1955,3,U.S. Steel,3250.4,195.4
3,1955,4,General Electric,2959.1,212.6
4,1955,5,Esmark,2510.8,19.1


In [461]:
#Drop all rows with missing informaiton in case
df=data_file.dropna(how='any')
df.count()

Year                     25500
Rank                     25500
Company                  25500
Revenue (in millions)    25500
Profit (in millions)     25500
dtype: int64

In [463]:
# Found the total rows of this df
Total_row_profit = len(df)
print(f"There are total number of {Total_row_profit} rows")

There are total number of 25500 rows


In [465]:
# Changed name Profit (in millions) to profit for conveniency  
# Used df['profit'] = pd.to_numeric(df['profit']) to find out what is causing a issue - found that there are N.A value
# rename Profit(in millions) to profit
df=df.rename(columns={'Profit (in millions)' : 'profit', 'Revenue (in millions)' : 'revenue'})

In [467]:
#Find the types of the profit and changed column name (Checked that this is non-numeric)
df.dtypes

Year         int64
Rank         int64
Company     object
revenue    float64
profit     float64
dtype: object

In [469]:
# Drop non-numeric value by using coerce 
df['profit'] = pd.to_numeric(df['profit'],errors='coerce')
cleaned_df=df.dropna(subset=['profit'])
cleaned_df

,Year,Rank,Company,revenue,profit
0,1955,1,General Motors,9823.5,806.0
1,1955,2,Exxon Mobil,5661.4,584.8
2,1955,3,U.S. Steel,3250.4,195.4
3,1955,4,General Electric,2959.1,212.6
4,1955,5,Esmark,2510.8,19.1
...,...,...,...,...,...
25495,2005,496,Wm. Wrigley Jr.,3648.6,493.0
25496,2005,497,Peabody Energy,3631.6,175.4
25497,2005,498,Wendy's International,3630.4,57.8
25498,2005,499,Kindred Healthcare,3616.6,70.6


In [471]:
#Check whether our df types got changed after drop non-numeric value from profit
cleaned_df.dtypes

Year         int64
Rank         int64
Company     object
revenue    float64
profit     float64
dtype: object

In [552]:
#Print total number of cleaned rows
Cleaned_Total_row_profit = len(cleaned_df)
print(f"There are total number of cleaned {Cleaned_Total_row_profit} rows")

There are total number of cleaned 25131 rows


In [553]:
#Save DF as Json file named "data2.json"
cleaned_df.to_json("data2.json")

In [554]:
import json
import os
from pandas.io.json import json_normalize 

In [555]:
# Read Jsonfile
filepath = os.path.join("data2.json")
with open(filepath) as jsonfile:
    json = json.load(jsonfile)

In [556]:
#Json to dataframe
json_df = pd.DataFrame.from_dict(json, orient="columns")

# In order to remove index column json_df.set_index('Year', inplace=True)


print(json_df.nlargest(20,['profit']))

       Year  Rank                 Company   revenue   profit
25001  2005     2             Exxon Mobil  270772.0  25330.0
22001  1999     2              Ford Motor  144416.0  22071.0
24501  2004     2             Exxon Mobil  213199.0  21510.0
24507  2004     8               Citigroup   94713.0  17853.0
23000  2001     1             Exxon Mobil  210392.0  17720.0
25007  2005     8               Citigroup  108276.0  17046.0
25004  2005     5        General Electric  152363.0  16593.0
23501  2002     2             Exxon Mobil  191581.0  15320.0
24005  2003     6               Citigroup  100789.0  15276.0
24504  2004     5        General Electric  134187.0  15002.0
25017  2005    18   Bank of America Corp.   63324.0  14143.0
23506  2002     7               Citigroup  112022.0  14126.0
24004  2003     5        General Electric  131698.0  14118.0
23505  2002     6        General Electric  125913.0  13684.0
23005  2001     6               Citigroup  111826.0  13519.0
25005  2005     6       

In [562]:
#writing sql
from sqlalchemy import create_engine

In [563]:
#Log into pgadmin and make table by using SQL quiry
rds_connection_string = "postgres:chldudwnd818@localhost:5432/SADA"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [568]:
#Check the table
engine.table_names()

['highest_profit']

In [571]:
#Use pandas to load dataframe into database
json_df.to_sql(name='highest_profit', con=engine, if_exists='append', index=False)


In [605]:
#Check the table include values
pd.read_sql_table('highest_profit', con=engine)

,Year,Rank,Company,revenue,profit
0,1955,1,General Motors,9823.5,806.0
1,1955,2,Exxon Mobil,5661.4,584.8
2,1955,3,U.S. Steel,3250.4,195.4
3,1955,4,General Electric,2959.1,212.6
4,1955,5,Esmark,2510.8,19.1
...,...,...,...,...,...
100519,2005,496,Wm. Wrigley Jr.,3648.6,493.0
100520,2005,497,Peabody Energy,3631.6,175.4
100521,2005,498,Wendy's International,3630.4,57.8
100522,2005,499,Kindred Healthcare,3616.6,70.6


In [606]:
# Run the squiry quiry and find the TOP 20 profit 
pd.read_sql_query('SELECT DISTINCT * from highest_profit ORDER by profit DESC LIMIT 20', con=engine)

,Year,Rank,Company,revenue,profit
0,2005,2,Exxon Mobil,270772.0,25330.0
1,1999,2,Ford Motor,144416.0,22071.0
2,2004,2,Exxon Mobil,213199.0,21510.0
3,2004,8,Citigroup,94713.0,17853.0
4,2001,1,Exxon Mobil,210392.0,17720.0
5,2005,8,Citigroup,108276.0,17046.0
6,2005,5,General Electric,152363.0,16593.0
7,2002,2,Exxon Mobil,191581.0,15320.0
8,2003,6,Citigroup,100789.0,15276.0
9,2004,5,General Electric,134187.0,15002.0
